In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from collections import Counter
import re
import jieba, jieba.analyse
import cn2an

In [2]:
jieba.set_dictionary('./jieba_data/dict.txt.big.txt')

In [3]:
jieba.load_userdict('jieba_data/mydict.txt')

Building prefix dict from C:\Users\CEB10202\PycharmProjects\pythonProject4\jieba_data\dict.txt.big.txt ...
Loading model from cache C:\Users\CEB10202\AppData\Local\Temp\jieba.u22d3d2a520f784251f61b3db4beb3030.cache
Loading model cost 1.127 seconds.
Prefix dict has been built successfully.


In [4]:
with open(file='./jieba_data/stop_words.txt', mode='r', encoding='utf-8') as file:      
     stop_words = file.read().split('\n')
stop_words

['the',
 'of',
 'is',
 'and',
 'to',
 '＄',
 'HD9240',
 'in',
 'that',
 'we',
 'for',
 'an',
 'are',
 'by',
 'be',
 'as',
 'on',
 'with',
 'can',
 'if',
 'from',
 'which',
 'you',
 'it',
 'this',
 'then',
 'at',
 'have',
 'all',
 'not',
 'one',
 'has',
 'or',
 'that',
 '的',
 '了',
 '和',
 '是',
 '就',
 '都',
 '而',
 '及',
 '與',
 '著',
 '或',
 '一個',
 '沒有',
 '我們',
 '你們',
 '妳們',
 '他們',
 '她們',
 '是否',
 '❤',
 '🧡',
 '💛',
 '💚',
 '',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '?',
 '_',
 '“',
 '”',
 '、',
 '。',
 '《',
 '》',
 '一',
 '一些',
 '一何',
 '一切',
 '一则',
 '一方面',
 '一旦',
 '一来',
 '一样',
 '一般',
 '一转眼',
 '万一',
 '上',
 '上下',
 '下',
 '不',
 '不仅',
 '不但',
 '不光',
 '不单',
 '不只',
 '不外乎',
 '不如',
 '不妨',
 '不尽',
 '不尽然',
 '不得',
 '不怕',
 '不惟',
 '不成',
 '不拘',
 '不料',
 '不是',
 '不比',
 '不然',
 '不特',
 '不独',
 '不管',
 '不至于',
 '不若',
 '不论',
 '不过',
 '不问',
 '与',
 '与其',
 '与其说',
 '与否',
 '与此同时',
 '且',
 '且不说',
 '且说',
 '两者',
 '个',
 '个别',
 '临',
 '为',
 '为了',
 '为什么',
 '为何',
 '为止',
 '为此',
 '为着',
 '乃',
 '乃至',
 '乃至于',
 '么',
 '之',
 '之一'

In [5]:
# url ='mongodb://root:root@***.***.***.***:27017/?authSource=admin&readPreference=primary&appname=MongoDB%20Compass&ssl=false'
# client = MongoClient(url)
client = MongoClient('127.0.0.1',27017)
# collection = client["Recipe_OriginData"]["cook1cook"]
collection = client['food']['cook1cook']
data = collection.find()
df = pd.DataFrame(data)
df.head()

,_id,Recipeid,CategoryName,SubCategoryID,SubCategoryName,RecipeName,Introduction,AuthorID,Author,Servings,CookingTime,Ingredient,CookingSteps,LikeStat,ContentEyes,RecipeURL
0,6067334bf52359e259b574bf,50992,異國料理,28,港式,會拉絲的|馬鈴薯芝士煎餅,軟綿綿入口即化的 \r馬鈴薯芝士煎餅\r材料簡單！而且不需要用油炸 \r裡面包著滿滿的Moz...,168267,Daruma Chan,3-4人,15分鐘內,"{'馬鈴薯': '3〜4個', '糖': '3湯匙', '粘米粉': '4湯匙', 'Moz...","[1首先\r\n把馬鈴薯去皮切片狀\r\n然後蒸大約30分鐘至熟透。, 2然後把馬鈴薯壓成泥...",1,89,NaN
1,6067334cf52359e259b574c0,50971,異國料理,28,港式,酥炸蒜香蝦醬排骨(氣炸版),酥炸蒜香蝦醬排骨是一道鹹香惹味的菜式 ，只要用上蕃薯粉作炸粉，會有更酥脆既效果，用埋氣炸鍋幫...,yeungma,YeungMa,3-4人,30分鐘內,"{'新鮮/急凍排骨': '1磅', '蕃薯粉/生粉/麵粉': '適量'}","[1急凍排骨先解凍後洗淨。, 2排骨放入已加入1大湯匙的鹽於水中浸約半小時（剔除雪味），之後...",1,125,NaN
2,6067334df52359e259b574c1,50940,異國料理,28,港式,焗南瓜籽,愛吃南瓜🎃的我，最近上食養課才知道原來啲核可以自己烘乾來吃，就是南瓜籽🤣\r烤得脆脆的還可以...,168200,Eunice Uno Cheung,1-2人,30分鐘內,{'南瓜': '1個'},"[1用匙羹把南瓜囊內的籽刮出來後洗淨及印乾表面水份, 2用鹽水浸1-2小時, 3焗盤鋪上錫紙...",0,43,NaN
3,60673352f52359e259b574c2,50959,異國料理,28,港式,雪耳南瓜粥,買了足足一個月先第一次用的燜燒鍋😅\r煲了個雪耳南瓜粥作第二朝早餐,168200,Eunice Uno Cheung,1-2人,30分鐘內,"{'雪耳': '1朵', '迷你南瓜': '1個', '米': '半杯'}","[1雪耳稍浸洗, 南瓜洗淨切開去囊去籽 (連皮用), 2燜燒鍋大火燒開約500ml水, 水滾...",0,37,NaN
4,60673353f52359e259b574c3,50975,異國料理,28,港式,中式甜品-蓮子百合燉桃膠湯水【滋陰養顏，養血安神】,【食療功效】\r清熱解毒、生津止渴、益氣、養血、養心、安神\r【適宜人群】\r一般人群均可食...,171004,Eva Tang,1-2人,1小時以上,"{'桃膠': '15g', '紅棗': '4粒', '桂圓': '10g', '蓮子': '...",[1桃膠加入大量清水浸泡 6小時(或過夜)，浸發後挑走黑色雜質，掰開成小份，沖洗乾淨瀝水備用...,0,220,NaN


In [6]:
l = []
df2 = df['Ingredient']
for i in df2.values: 
    for w in i.keys():
        l.append(w)
        l.append(i[w])
l

['馬鈴薯',
 '3〜4個',
 '糖',
 '3湯匙',
 '粘米粉',
 '4湯匙',
 'Mozarella 水牛芝士',
 '適量',
 '新鮮/急凍排骨',
 '1磅',
 '蕃薯粉/生粉/麵粉',
 '適量',
 '南瓜',
 '1個',
 '雪耳',
 '1朵',
 '迷你南瓜',
 '1個',
 '米',
 '半杯',
 '桃膠',
 '15g',
 '紅棗',
 '4粒',
 '桂圓',
 '10g',
 '蓮子',
 '15g',
 '百合',
 '15g',
 '冰糖',
 '適量',
 '熱水',
 '注滿',
 '羊肚菌',
 '4粒',
 '姬松茸',
 '10個',
 '合桃',
 '20g',
 '栗子',
 '5粒',
 '粟米',
 '1條',
 '紅蘿蔔',
 '1/2條',
 '蜜棗',
 '2粒',
 '陳皮',
 '1/2角',
 '清水',
 '1200-1500m',
 '白蘿蔔',
 '十片',
 '魚肉',
 '10蚊',
 '汁料',
 '',
 '魚露',
 '1匙',
 '醬油',
 '1匙',
 '糖',
 '1/2匙',
 '粟粉',
 '少許',
 '水',
 '3匙',
 '姬松茸',
 '20g',
 '茶樹菇',
 '20g',
 '猴頭菇',
 '25g',
 '竹笙',
 '20g',
 '豬腱肉',
 '129g',
 '新鮮雞',
 '1/2 pc',
 '無花果乾',
 '2 粒',
 '薑片',
 '2 片',
 '清水',
 '2500ml',
 '豬小排',
 '1公斤',
 '花雕酒',
 '2-5大匙',
 '醬油',
 '3大匙',
 '蒜末',
 '0-5大匙',
 '砂糖',
 '25g',
 '白胡椒粉',
 '0-5小匙',
 '全蛋',
 '1顆',
 '太白粉',
 '0-5大匙',
 '柳橙汁',
 '300ml',
 '檸檬汁',
 '15ml',
 '地瓜粉',
 '200g',
 '芡水',
 '2大匙',
 '白芝麻',
 '少許',
 '五指毛桃',
 '30g',
 '蓮子',
 '30g',
 '百合',
 '15g',
 '茨實',
 '15g',
 '無花果',
 '2個',
 '瘦肉',
 '300g',
 '節瓜',
 '1個',
 '海

In [ ]:
# fileAllLines = []
# for str in name:
#     if str not in stop_words:
#         seg = jieba.cut(str)
#         fileAllLines.append(seg)
#         print(list(seg))

In [7]:
s = ''
for w in l:
    s += w
seg = jieba.lcut(s)
seg

['馬鈴薯',
 '3',
 '〜',
 '4',
 '個',
 '糖',
 '3',
 '湯匙',
 '粘',
 '米粉',
 '4',
 '湯匙',
 'Mozarella',
 ' ',
 '水牛',
 '芝士',
 '適量',
 '新鮮',
 '/',
 '急凍',
 '排骨',
 '1',
 '磅',
 '蕃薯粉',
 '/',
 '生粉',
 '/',
 '麵',
 '粉',
 '適量',
 '南瓜',
 '1',
 '個雪耳',
 '1',
 '朵',
 '迷你',
 '南瓜',
 '1',
 '個',
 '米',
 '半杯',
 '桃膠',
 '15g',
 '紅棗',
 '4',
 '粒',
 '桂圓',
 '10g',
 '蓮子',
 '15g',
 '百合',
 '15g',
 '冰糖',
 '適量',
 '熱水',
 '注滿',
 '羊肚',
 '菌',
 '4',
 '粒',
 '姬松茸',
 '10',
 '個合桃',
 '20g',
 '栗子',
 '5',
 '粒',
 '粟米',
 '1',
 '條',
 '紅蘿蔔',
 '1',
 '/',
 '2',
 '條',
 '蜜棗',
 '2',
 '粒陳皮',
 '1',
 '/',
 '2',
 '角',
 '清水',
 '1200-1500',
 'm',
 '白蘿蔔',
 '十片',
 '魚肉',
 '10',
 '蚊汁料',
 '魚露',
 '1',
 '匙',
 '醬油',
 '1',
 '匙',
 '糖',
 '1',
 '/',
 '2',
 '匙',
 '粟粉',
 '少許',
 '水',
 '3',
 '匙',
 '姬松茸',
 '20g',
 '茶樹菇',
 '20g',
 '猴頭菇',
 '25g',
 '竹笙',
 '20g',
 '豬腱',
 '肉',
 '129g',
 '新鮮',
 '雞',
 '1',
 '/',
 '2',
 ' ',
 'pc',
 '無花果',
 '乾',
 '2',
 ' ',
 '粒',
 '薑片',
 '2',
 ' ',
 '片',
 '清水',
 '2500ml',
 '豬小排',
 '1',
 '公斤',
 '花雕酒',
 '2-5',
 '大匙',
 '醬油',
 '3',
 '大匙',
 '蒜末',
 '0-5',


In [11]:
word_list = [k for k in seg if len(k) > 1 and k not in stop_words]
word_list

['馬鈴薯',
 '湯匙',
 '米粉',
 '湯匙',
 '水牛',
 '芝士',
 '適量',
 '新鮮',
 '急凍',
 '排骨',
 '蕃薯粉',
 '生粉',
 '適量',
 '南瓜',
 '個雪耳',
 '迷你',
 '南瓜',
 '半杯',
 '桃膠',
 '15g',
 '紅棗',
 '桂圓',
 '10g',
 '蓮子',
 '15g',
 '百合',
 '15g',
 '冰糖',
 '適量',
 '熱水',
 '注滿',
 '羊肚',
 '姬松茸',
 '10',
 '個合桃',
 '20g',
 '栗子',
 '粟米',
 '紅蘿蔔',
 '蜜棗',
 '粒陳皮',
 '清水',
 '1200-1500',
 '白蘿蔔',
 '十片',
 '魚肉',
 '10',
 '蚊汁料',
 '魚露',
 '醬油',
 '粟粉',
 '少許',
 '姬松茸',
 '20g',
 '茶樹菇',
 '20g',
 '猴頭菇',
 '25g',
 '竹笙',
 '20g',
 '豬腱',
 '129g',
 '新鮮',
 '無花果',
 '薑片',
 '清水',
 '2500ml',
 '豬小排',
 '公斤',
 '花雕酒',
 '2-5',
 '大匙',
 '醬油',
 '大匙',
 '蒜末',
 '0-5',
 '大匙',
 '砂糖',
 '25g',
 '白胡椒粉',
 '0-5',
 '小匙',
 '全蛋',
 '太白粉',
 '0-5',
 '大匙',
 '柳橙汁',
 '300ml',
 '檸檬汁',
 '15ml',
 '地瓜',
 '200g',
 '芡水',
 '大匙',
 '白芝麻',
 '少許',
 '五指毛桃',
 '30g',
 '蓮子',
 '30g',
 '百合',
 '15g',
 '茨實',
 '15g',
 '無花果',
 '瘦肉',
 '300g',
 '節瓜',
 '海螺',
 '25g',
 '日月',
 '20g',
 '淮山',
 '20g',
 '瘦肉',
 '300g',
 '清水',
 '1200-1500',
 '雞腿肉',
 '一塊',
 '白蘿蔔',
 '600',
 '數片',
 '蔥段',
 '適量',
 '甜麵醬',
 '一匙',
 '蒜蓉',
 '湯匙',
 '急凍',
 '牛柳',
 '粒

In [12]:
segSaveFile = './jieba_data/segDone.txt'
with open(segSaveFile, 'wb') as saveFile:
    for i in range(len(word_list)):
        #words = seg[i][0].encode('utf-8')
        words = word_list[i].encode('utf-8')
        if len(words) > 0:
            saveFile.write(words)
            saveFile.write('\n'.encode())

In [20]:
stg = str(seg)
tf = jieba.analyse.extract_tags(stg, topK=200)
print(tf)

['湯匙', '適量', '少許', '茶匙', '大匙', '蒜頭', '10', '雞蛋', '胡椒粉', '生抽', '生粉', '洋蔥', '麻油', '小匙', '牛油', '隨意', '醬油', '蒜蓉', '1-5', '紅蘿蔔', '老抽', '半個', '冰糖', '蠔油', '薑片', '20', '豬肉', '鹽適量', '冬菇', '醃料', '米酒', '100g', '急凍', '20g', '粟米', '辣椒', '雞粉', '芝士', '50g', '15', '30g', '砂糖', '毫升', '200g', '100', '雞湯', '30', '12', '2-3', '黑胡椒', '蔥花', '蜜棗', '200', '蕃茄', '300g', '切碎', '調味料', '半斤', '橄欖油', '低筋麵粉', '150g', '新鮮', '粟粉', '切粒', '紅棗', '牛奶', '50', '1-2', '10g', '豆腐', '300', '40g', '紹興酒', '15g', '黑椒', '乾蔥', '40', '3-4', '60g', '蛋黃', '150', '瘦肉', '無鹽', '排骨', '薯仔', '蜜糖', '一個', '半條', '250g', '青蔥', '半碗', '80g', '雞翼', '芫茜', '蝦米', '米粉', '豉油', '紅椒', '鹽巴', '清水', '檸檬', '250', '粉絲', '蔥頭', '糯米粉', '一隻', '香油', '鮮奶', '一條', '無花果', '500g', '薑絲', '泡打粉', '淮山', '蝦仁', '魚露', '半隻', '500', '切絲', '椰漿', '蒜末', '白芝麻', '200ml', '4-5', '熱水', '八角', '25g', '400', '白蘿蔔', '3g', '100ml', '白胡椒粉', '海鹽', '克水', '25', '罐頭', '80', '杞子', '蛋白', '材料', '南瓜', '蓮子', '切片', '60', '中筋麵粉', '豆豉', '酵母', '蘋果', '臘腸', '牛肉', '五香粉', '高筋麵粉', '5g', '檸檬汁', '紅蔥頭', '椰菜', '

In [ ]:
# seg_counter = Counter(word_list)
# seg_counter